In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib.cm as cm
from scipy.interpolate import interp1d
from EOF import EOF
import datetime 
from scipy.interpolate import RectBivariateSpline

In [2]:
from Dycore import Dycore
def read_all_var(pr_start, pr_to, pr_leap, start, end, internal_day, total_file):
    total_pr = int((pr_to - pr_start) / pr_leap)
    total_day = int(total_file * internal_day)
    
    qv_shape = (total_pr, total_day, 20, 64, 128)
    qv_diff_shape = (total_pr, total_day, 20, 64, 128)
    u_shape = (total_pr, total_day, 20, 64, 128)
    v_shape = (total_pr, total_day, 20, 64, 128)
    t_shape = (total_pr, total_day, 20, 64, 128)
    p_shape = (total_pr, total_day, 20, 64, 128)
    ps_shape = (total_pr, total_day, 20, 64, 128)
    p_half_shape = (total_pr, total_day, 21, 64, 128)
    
    qv = np.zeros(qv_shape)
    qv_diff = np.zeros(qv_diff_shape)
    u = np.zeros(u_shape)
    v = np.zeros(v_shape)
    t = np.zeros(t_shape)
    p = np.zeros(p_shape)
    ps = np.zeros(ps_shape)
    p_half = np.zeros(p_half_shape)
    
    a = 0
    for pr in range(pr_start, pr_to, pr_leap):
        k = 0
        for day in range(start, end, internal_day):
            print(day)
            if pr == 8 and day == 7500:
                print("PR=8, 7500day file miss")
            else:
                file = f"HSt42_{pr}/RH80_PR{pr}_20000day_startfrom_{day}day_final.dat"
                dycore = Dycore(file)
                
                start_index = k * 100
                end_index = (k + 1) * 100
                
                qv[a, start_index:end_index, :, :, :] = dycore.qv
                qv_diff[a, start_index:end_index, :, :, :] = dycore.qv_diff
                u[a, start_index:end_index, :, :, :] = dycore.u
                v[a, start_index:end_index, :, :, :] = dycore.v
                t[a, start_index:end_index, :, :, :] = dycore.t
                ps[a, start_index:end_index, :, :, :] = dycore.ps
                p[a, start_index:end_index, :, :, :] = dycore.p
                p_half[a, start_index:end_index, :, :, :] = dycore.p_half
                k += 1
        a += 1
    return qv, qv_diff, u, v, t, p, ps, p_half

In [3]:
pr_start = 10
pr_leap = 10
pr_to = 60
total_pr = (pr_to - pr_start) / pr_leap

start = 500
end   = 20000
internal_day = 100
total_day = int((end -start)/internal_day) * internal_day

total_file = int((end - start) / internal_day)




print("done")


done


In [4]:
### read variables ###
u_file       = h5py.File("PR10_50_0_20000day_u.h5", "r")
# w_file       = h5py.File("PR10_50_0_20000day_w.h5", "r")

# qv_file      = h5py.File("PR10_50_0_20000day_qv.h5", "r")
# qv_diff_file = h5py.File("PR10_50_0_20000day_qv_diff.h5", "r")
# p_file       = h5py.File("PR10_50_0_20000day_p.h5", "r")
# ps_file      = h5py.File("PR10_50_0_20000day_ps.h5", "r")
# p_half_file  = h5py.File("PR10_50_0_20000day_p_half.h5", "r")

# z_full_file  = h5py.File("PR10_50_0_20000day_z_full.h5", "r")



print("---1---")
u_origin       = np.asarray(u_file["u"][:,int(start):end,:,:,:])
# qv_origin       = np.asarray(qv_file["qv"][:,int(start):end,:,:,:])
# print("---2---")
# qv_diff_origin = np.asarray(qv_diff_file["qv_diff"][:,int(start):end,:,:,:])
# print("---3---")
# p_origin       = np.asarray(p_file["p"][:,int(start):end,:,:,:])
# print("---4---")
# ps_origin      = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])
# print("---5---")
# p_half_origin  = np.asarray(p_half_file["p_half"][:,int(start):end,:,:,:])
# print("---6---")

# w_origin       = np.asarray(w_file["w"][:,int(start):end,:,:,:])
# z_full_origin       = np.asarray(z_full_file["z_full"][:,int(start):end,:,:,:])


print("done")

u_file.close()
# qv_file.close()
# qv_diff_file.close()
# p_file.close()
# ps_file.close()
# p_half_file.close()
# w_file.close()
# z_full_file.close()

v_file       = h5py.File("PR10_50_0_20000day_v.h5", "r")
v_origin       = np.asarray(v_file["v"][:,int(start):end,:,:,:])
t_file       = h5py.File("PR10_50_0_20000day_t.h5", "r")
t_origin       = np.asarray(t_file["t"][:,int(start):end,:,:,:])

v_file.close()
t_file.close()

print("done")



---1---
done
done


In [5]:
u = u_origin
# qv = qv_origin
# qv_diff = qv_diff_origin
# p = p_origin
# ps = ps_origin
# p_half = p_half_origin
# w = w_origin
# z_full = z_full_origin
v = v_origin
t = t_origin

In [6]:
print(u.shape)

(5, 100, 20, 64, 128)


# time mean T for EHF

In [ ]:
Ttm = np.nanmean(t, axis=1)
with h5py.File('PR10_50_500_20000day_t_time_mean.h5','w') as Ttm_file :
    Ttm_file.create_dataset('t',data=Ttm)
    # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# time mean, zonal mean

In [7]:
# u_mean = np.nanmean(u, axis=(1,4))
# qv_mean = np.nanmean(qv, axis=(1,4))
# qv_diff_mean = np.nanmean(qv_diff, axis=(1,4))
# p_mean = np.nanmean(p, axis=(1,4))
# ps_mean = np.nanmean(ps, axis=(1,4))
# p_half_mean = np.nanmean(p_half, axis=(1,4))
# w_mean = np.nanmean(w, axis=(1,4))
# z_full_mean = np.nanmean(z_full, axis=(1,4))
# v_mean = np.nanmean(v, axis=(1,4))
# t_mean = np.nanmean(t, axis=(1,4))
# print("done")


In [8]:
# # you would get (5,20,64)
# str = "time_zonal_mean"
# with h5py.File(str+'PR10_50_0_20000day_u.h5','w') as u_file :
#     u_file.create_dataset('u',data=u_mean)
#     # u = np.asarray(u_file["u"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_qv.h5','w') as qv_file :
#     qv_file.create_dataset('qv',data=qv_mean)
#     # qv = np.asarray(qv_file["qv"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_qv_diff.h5','w') as qv_diff_file :
#     qv_diff_file.create_dataset('qv_diff',data=qv_diff_mean)
#     # qv_diff = np.asarray(qv_diff_file["qv_diff"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_v.h5','w') as v_file :
#     v_file.create_dataset('v',data=v_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_t.h5','w') as t_file :
#     t_file.create_dataset('t',data=t_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_w.h5','w') as w_file :
#     w_file.create_dataset('w',data=w_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# with h5py.File(str+'PR10_50_0_20000day_p.h5','w') as p_file :
#     p_file.create_dataset('p',data=p_mean)
#     # p = np.asarray(p_file["p"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_ps.h5','w') as ps_file :
#     ps_file.create_dataset('ps',data=ps_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_p_half.h5','w') as p_half_file :
#     p_half_file.create_dataset('p_half',data=p_half_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_20000day_z_full.h5','w') as z_full_file :
#     z_full_file.create_dataset('z_full',data=z_full_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# print("done")

In [9]:
# str = "time_zonal_mean"

# u_test      = h5py.File(str+"PR10_50_0_20000day_u.h5", "r")
# u_origin       = np.asarray(u_test["u"][:,:,:])


In [10]:
# print(u_origin.shape)

# zonal mean

In [11]:
# u_mean = np.nanmean(u, axis=(4))
# qv_mean = np.nanmean(qv, axis=(4))
# qv_diff_mean = np.nanmean(qv_diff, axis=(4))
# p_mean = np.nanmean(p, axis=(4))
# ps_mean = np.nanmean(ps, axis=(4))
# p_half_mean = np.nanmean(p_half, axis=(4))
# w_mean = np.nanmean(w, axis=(4))
# z_full_mean = np.nanmean(z_full, axis=(4))
# v_mean = np.nanmean(v, axis=(4))
# t_mean = np.nanmean(t, axis=(4))



In [12]:
# # you would get (5,20,64)
# str = "tmp_zonal_mean"
# with h5py.File(str+'PR10_50_0_10000day_u.h5','w') as u_file :
#     u_file.create_dataset('u',data=u_mean)
#     # u = np.asarray(u_file["u"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_qv.h5','w') as qv_file :
#     qv_file.create_dataset('qv',data=qv_mean)
#     # qv = np.asarray(qv_file["qv"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_qv_diff.h5','w') as qv_diff_file :
#     qv_diff_file.create_dataset('qv_diff',data=qv_diff_mean)
#     # qv_diff = np.asarray(qv_diff_file["qv_diff"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_v.h5','w') as v_file :
#     v_file.create_dataset('v',data=v_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_t.h5','w') as t_file :
#     t_file.create_dataset('t',data=t_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_w.h5','w') as w_file :
#     w_file.create_dataset('w',data=w_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# with h5py.File(str+'PR10_50_0_10000day_p.h5','w') as p_file :
#     p_file.create_dataset('p',data=p_mean)
#     # p = np.asarray(p_file["p"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_ps.h5','w') as ps_file :
#     ps_file.create_dataset('ps',data=ps_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_p_half.h5','w') as p_half_file :
#     p_half_file.create_dataset('p_half',data=p_half_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_0_10000day_z_full.h5','w') as z_full_file :
#     z_full_file.create_dataset('z_full',data=z_full_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# print("finally done")

In [13]:
# # you would get (5,20,64)
# str = "tmp_zonal_mean"
# with h5py.File(str+'PR10_50_10000_20000day_u.h5','w') as u_file :
#     u_file.create_dataset('u',data=u_mean)
#     # u = np.asarray(u_file["u"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_qv.h5','w') as qv_file :
#     qv_file.create_dataset('qv',data=qv_mean)
#     # qv = np.asarray(qv_file["qv"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_qv_diff.h5','w') as qv_diff_file :
#     qv_diff_file.create_dataset('qv_diff',data=qv_diff_mean)
#     # qv_diff = np.asarray(qv_diff_file["qv_diff"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_v.h5','w') as v_file :
#     v_file.create_dataset('v',data=v_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_t.h5','w') as t_file :
#     t_file.create_dataset('t',data=t_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_w.h5','w') as w_file :
#     w_file.create_dataset('w',data=w_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# with h5py.File(str+'PR10_50_10000_20000day_p.h5','w') as p_file :
#     p_file.create_dataset('p',data=p_mean)
#     # p = np.asarray(p_file["p"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_ps.h5','w') as ps_file :
#     ps_file.create_dataset('ps',data=ps_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_p_half.h5','w') as p_half_file :
#     p_half_file.create_dataset('p_half',data=p_half_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])


# with h5py.File(str+'PR10_50_10000_20000day_z_full.h5','w') as z_full_file :
#     z_full_file.create_dataset('z_full',data=z_full_mean)
#     # ps = np.asarray(ps_file["ps"][:,int(start):end,:,:,:])

# print("finally done")

# load test

In [6]:
# str = "tmp_zonal_mean"
# u_file1      = h5py.File(str+"PR10_50_0_10000day_u.h5", "r")
# u_file2      = h5py.File(str+"PR10_50_10000_20000day_u.h5", "r")

# u1       = np.asarray(u_file1["u"][:,:,:,:])
# u2       = np.asarray(u_file2["u"][:,:,:,:])
# ####################################################################
# v_file1      = h5py.File(str+"PR10_50_0_10000day_v.h5", "r")
# v_file2      = h5py.File(str+"PR10_50_10000_20000day_v.h5", "r")

# v1       = np.asarray(v_file1["v"][:,:,:,:])
# v2       = np.asarray(v_file2["v"][:,:,:,:])
# ####################################################################
# t_file1      = h5py.File(str+"PR10_50_0_10000day_t.h5", "r")
# t_file2      = h5py.File(str+"PR10_50_10000_20000day_t.h5", "r")

# t1       = np.asarray(t_file1["t"][:,:,:,:])
# t2       = np.asarray(t_file2["t"][:,:,:,:])



In [7]:
# print(u1.shape)

(5, 10000, 20, 64)


In [8]:
# print(int(end-start))

100


In [9]:
# Uzm = np.append(u1, u2, axis=1)
# Vzm = np.append(v1, v2, axis=1)
# Tzm = np.append(t1, t2, axis=1)

# print(Uzm.shape)

(5, 20000, 20, 64)


In [13]:
# total_day = int(end-start)
# u_prime = np.zeros(((((5, total_day, 20, 64, 128)))))
# v_prime = np.zeros(((((5, total_day, 20, 64, 128)))))
# t_prime = np.zeros(((((5, total_day, 20, 64, 128)))))


# for a in range(0, 5):
#     for j in range(total_day):
#         for i in range(128):    
#             u_prime[a,j,:,:,i] = u[a,j,:,:,i] - Uzm[a,int(j+start)] # np.rollaxis((np.rollaxis(u[a], 3, 0) - np.nanmean(u[a], axis=3)), 0, 4)
#             v_prime[a,j,:,:,i] = v[a,j,:,:,i] - Vzm[a,int(j+start)] # np.rollaxis((np.rollaxis(v[a], 3, 0) - np.nanmean(v[a], axis=3)), 0, 4)
#             t_prime[a,j,:,:,i] = t[a,j,:,:,i] - Tzm[a,int(j+start)] # np.rollaxis((np.rollaxis(t[a], 3, 0) - np.nanmean(t[a], axis=3)), 0, 4)
    

In [ ]:
# with h5py.File('PR10_50_500_20000day_u_prime.h5','w') as u_prime_file :
#     u_prime_file.create_dataset('u_prime',data=u_prime)

# with h5py.File('PR10_50_500_20000day_v_prime.h5','w') as v_prime_file :
#     v_prime_file.create_dataset('v_prime',data=v_prime)

# with h5py.File('PR10_50_500_20000day_t_prime.h5','w') as t_prime_file :
#     t_prime_file.create_dataset('t_prime',data=t_prime)